In [1]:
import os
import numpy as np
import math
import matplotlib.pyplot as plt
from tqdm import tqdm
import pandas as pd
import xarray as xr
import h5netcdf
%matplotlib inline

In [2]:
# loading dataset from .dat files
# === setting ===
BASE_PATH = "./data/"
INPUT_PATHS = {
    "up_x": os.path.join(BASE_PATH, "fermi_line_kx_ky_Xpoint_up/"),
    "up_ganma": os.path.join(BASE_PATH, "fermi_line_kx_ky_ganma_up/"),
    "down_ganma": os.path.join(BASE_PATH, "fermi_line_kx_ky_ganma_down/")
}
BZ_FILE_PATH = os.path.join(BASE_PATH, "bz_line_ganma.dat")
OUTPUT_PATHS = {
    "up": os.path.join(BASE_PATH, "save_data/fermi_line_kx_ky_result_up/"),
    "down": os.path.join(BASE_PATH, "save_data/fermi_line_kx_ky_result_down/")
}
ROTATION_ANGLE = 45  # rotation (deg.)

# === maximum value of BZ===
bz_data = np.loadtxt(BZ_FILE_PATH)
TRANSLATION_AMOUNT = np.max(np.abs(bz_data))

# === functions ===
def load_data(file_path):
    """loading dataset. arg=path"""
    try:
        data = np.loadtxt(file_path)
        return data if data.size else np.empty((0, 2))
    except (OSError, ValueError):
        return np.empty((0, 2))

def parallel_translate(coords, translation_vector):
    return coords + translation_vector

def rotate_coordinates(coords, angle_degrees):
    angle_radians = np.radians(angle_degrees)
    rotation_matrix = np.array([[np.cos(angle_radians), -np.sin(angle_radians)],
                                [np.sin(angle_radians), np.cos(angle_radians)]])
    return np.dot(coords, rotation_matrix)

def process_and_save_data(filename):
    """save data for each files"""
    data_up_X = load_data(os.path.join(INPUT_PATHS["up_x"], filename))
    data_up_ganma = load_data(os.path.join(INPUT_PATHS["up_ganma"], filename))
    data_down_ganma = load_data(os.path.join(INPUT_PATHS["down_ganma"], filename))

    translations = [
        np.array([TRANSLATION_AMOUNT * 2, 0]),
        np.array([-TRANSLATION_AMOUNT * 2, 0]),
        np.array([0, TRANSLATION_AMOUNT * 2]),
        np.array([0, -TRANSLATION_AMOUNT * 2]),
        np.array([TRANSLATION_AMOUNT, TRANSLATION_AMOUNT]),
        np.array([-TRANSLATION_AMOUNT, TRANSLATION_AMOUNT]),
        np.array([TRANSLATION_AMOUNT, -TRANSLATION_AMOUNT]),
        np.array([-TRANSLATION_AMOUNT, -TRANSLATION_AMOUNT])
    ]

    translated_coords_up = [parallel_translate(data_up_X, t) for t in translations[:4]] if data_up_X.size else []
    translated_coords_X_up = [parallel_translate(data_up_ganma, t) for t in translations[4:]] if data_up_ganma.size else []
    all_translated_coords_up = np.vstack([data_up_X] + translated_coords_up + translated_coords_X_up) if data_up_X.size or data_up_ganma.size else np.empty((0, 2))
    rotated_coords_up = rotate_coordinates(all_translated_coords_up, ROTATION_ANGLE)

    translated_coords_X_down = [parallel_translate(data_down_ganma, t) for t in translations[4:]] if data_down_ganma.size else []
    all_translated_coords_down = np.vstack(translated_coords_X_down) if data_down_ganma.size else np.empty((0, 2))
    rotated_coords_down = rotate_coordinates(all_translated_coords_down, ROTATION_ANGLE)
    
    os.makedirs(os.path.join(OUTPUT_PATHS["up"]), exist_ok=True)
    os.makedirs(os.path.join(OUTPUT_PATHS["down"]), exist_ok=True)

    np.save(os.path.join(OUTPUT_PATHS["up"], filename.replace(".dat", ".npy")), rotated_coords_up)
    np.save(os.path.join(OUTPUT_PATHS["down"], filename.replace(".dat", ".npy")), rotated_coords_down)

# === main ===
# composition_list = os.listdir(INPUT_PATHS["up_x"])
# for filename in composition_list:
#     process_and_save_data(filename)


In [3]:
# image processing (mimicking SARPES data)
# --- setting ---
SIGMA = 4  # broadening width
PIXEL_SIZE = 255  # image sizes
BZ_MAX = TRANSLATION_AMOUNT  
RANGE_PIXEL = (BZ_MAX / 2 * 5) / math.sqrt(2)  # calc. region

# path setting
DATA_DIRS = {
    "input_up": os.path.join(BASE_PATH, "save_data/fermi_line_kx_ky_result_up"),
    "input_down": os.path.join(BASE_PATH, "save_data/fermi_line_kx_ky_result_down"),
    "position_up": os.path.join(BASE_PATH, "save_data/data_position_list/kx_ky_up"),
    "position_down": os.path.join(BASE_PATH, "save_data/data_position_list/kx_ky_down"),
    "figure": os.path.join(BASE_PATH, "save_data/figure/fermi_line_kxky_fig"),
    "gaussian_figure": os.path.join(BASE_PATH, "save_data/figure/gaussian"),
    "gaussian_up": os.path.join(BASE_PATH, f"save_data/gaussian/sigma{SIGMA}_up_npy_kx_ky"),
    "gaussian_down": os.path.join(BASE_PATH, f"save_data/gaussian/sigma{SIGMA}_down_npy_kx_ky"),
}

# --- functions ---
def load_npy(file_path):
    """ loading npy data. if it don't exists, return empty."""
    return np.load(file_path) if os.path.exists(file_path) else np.empty((0, 2))

def coordinates_to_pixels(data, x_min, x_max, y_min, y_max, pixel_size):
    """ coordination to pixels """
    if data.size == 0:
        return np.array([]).reshape(0, 2)
    
    pixel_x = ((data[:, 0] - x_min) / (x_max - x_min) * pixel_size).astype(int)
    pixel_y = ((data[:, 1] - y_min) / (y_max - y_min) * pixel_size).astype(int)
    return np.column_stack((pixel_x, pixel_y))

def generate_smooth_gradient(size, white_pixel_positions, sigma):
    """ broadening with gaussian blurring """
    image = np.zeros((size, size))
    x, y = np.meshgrid(np.arange(size), np.arange(size))
    
    for cx, cy in white_pixel_positions:
        d = np.sqrt((x - cx) ** 2 + (y - cy) ** 2)
        image += np.exp(-d ** 2 / (2.0 * sigma ** 2))

    # 画像の正規化
    return (image - image.min()) / (image.max() - image.min()) if image.max() > 0 else image

def process_files(filename):
    """ main process. """
    filename = filename.replace(".dat", "")
    
    # データ読み込み
    data_up = load_npy(os.path.join(DATA_DIRS["input_up"], f"{filename}.npy"))
    data_down = load_npy(os.path.join(DATA_DIRS["input_down"], f"{filename}.npy"))

    # 座標範囲設定
    x_min, x_max, y_min, y_max = -RANGE_PIXEL, RANGE_PIXEL, -RANGE_PIXEL, RANGE_PIXEL

    # ピクセル座標変換
    pixel_coords_up = coordinates_to_pixels(data_up, x_min, x_max, y_min, y_max, PIXEL_SIZE)
    pixel_coords_down = coordinates_to_pixels(data_down, x_min, x_max, y_min, y_max, PIXEL_SIZE)
    
    os.makedirs(os.path.join(DATA_DIRS["position_up"]), exist_ok=True)
    os.makedirs(os.path.join(DATA_DIRS["position_down"]), exist_ok=True)
    os.makedirs(os.path.join(DATA_DIRS["figure"]), exist_ok=True)

    # データ位置リストを保存
    np.save(os.path.join(DATA_DIRS["position_up"], f"{filename}_up.npy"), pixel_coords_up)
    np.save(os.path.join(DATA_DIRS["position_down"], f"{filename}_down.npy"), pixel_coords_down)

    # --- プロット作成 ---
    plt.scatter(pixel_coords_up[:, 0], pixel_coords_up[:, 1], color='red', label='up spin', s=5)
    plt.scatter(pixel_coords_down[:, 0], pixel_coords_down[:, 1], color='blue', label='down spin', s=5)
    plt.legend(loc='upper right')
    plt.title(filename, fontsize=20)
    plt.xlabel('kx', fontsize=20)
    plt.ylabel('ky', fontsize=20)
    plt.xlim(0, PIXEL_SIZE)
    plt.ylim(PIXEL_SIZE, 0)
    plt.gca().set_aspect('equal', adjustable='box')
    plt.savefig(os.path.join(DATA_DIRS["figure"], f"{filename}.png"))
    plt.close()

    # --- ガウシアン処理 ---
    if pixel_coords_up.size > 0:
        gradient_image_up = generate_smooth_gradient(PIXEL_SIZE, pixel_coords_up, SIGMA)
    else:
        gradient_image_up = np.zeros((PIXEL_SIZE, PIXEL_SIZE))

    if pixel_coords_down.size > 0:
        gradient_image_down = generate_smooth_gradient(PIXEL_SIZE, pixel_coords_down, SIGMA)
    else:
        gradient_image_down = np.zeros((PIXEL_SIZE, PIXEL_SIZE))
        
        
    os.makedirs(os.path.join(DATA_DIRS["gaussian_up"]), exist_ok=True)
    os.makedirs(os.path.join(DATA_DIRS["gaussian_down"]), exist_ok=True)
    os.makedirs(os.path.join(DATA_DIRS["gaussian_figure"]), exist_ok=True)

    np.save(os.path.join(DATA_DIRS["gaussian_up"], f"{filename}_up.npy"), gradient_image_up)
    np.save(os.path.join(DATA_DIRS["gaussian_down"], f"{filename}_down.npy"), gradient_image_down)

    # --- スムージング後の画像表示 ---
    combined_array = np.where(np.abs(gradient_image_up) > np.abs(gradient_image_down), gradient_image_up, -gradient_image_down)

    plt.imshow(combined_array, cmap="seismic", vmin=-1, vmax=1)
    plt.xticks([])
    plt.yticks([])
    plt.title(filename, fontsize=20)
    plt.savefig(os.path.join(DATA_DIRS["gaussian_figure"], f"{filename}.png"), bbox_inches='tight')
    plt.close()

# --- メイン処理 ---
# for filename in tqdm(composition_list):
#     process_files(filename)


In [6]:

# 定数の定義
SIGMA = 4  # 必要に応じて変更
INPUT_PATH_UP = DATA_DIRS["gaussian_up"]
INPUT_PATH_DOWN = DATA_DIRS["gaussian_down"]
OUTPUT_H5_PATH = os.path.join(BASE_PATH, "save_data/h5")


# ガリウム組成のリストを生成
Ga_composition = np.linspace(0, 100, 101, dtype=int) / 100

# スピン偏極率データの読み込み
spin_df = pd.read_csv("data/spin_polalization_article.csv", header=0, names=["spin_polarization"])

# ファイルリストの取得（エラーハンドリング付き）
def get_file_list(directory):
    try:
        return sorted(os.listdir(directory))
    except FileNotFoundError:
        print(f"Error: Directory not found - {directory}")
        return []

gaussian_list_up = get_file_list(INPUT_PATH_UP)
gaussian_list_down = get_file_list(INPUT_PATH_DOWN)

# データの読み込み

def load_gaussian_data(file_list, directory):
    """指定ディレクトリ内の NumPy ファイルを読み込んでリスト化"""
    return [np.load(os.path.join(directory, filename)) for filename in file_list]

up_data = load_gaussian_data(gaussian_list_up, INPUT_PATH_UP)
down_data = load_gaussian_data(gaussian_list_down, INPUT_PATH_DOWN)

# xarray Dataset の作成
data = xr.Dataset(
    {
        "up": (["Ga_comp", "kx", "ky"], up_data),
        "down": (["Ga_comp", "kx", "ky"], down_data),
    },
    coords={
        "Ga_comp": Ga_composition,
        "kx": np.arange(1, 256),
        "ky": np.arange(1, 256),
        "spin_polarization": np.array(spin_df["spin_polarization"]),
    },
    attrs={"material": "Co2MnGaGe"},
)


# データの保存
os.makedirs(OUTPUT_H5_PATH, exist_ok=True)
data.to_netcdf(f"{OUTPUT_H5_PATH}/data.h5", mode="w", engine="h5netcdf")
print(f"Data saved to {OUTPUT_H5_PATH}")

Data saved to ./data/save_data/h5
